<a href="https://colab.research.google.com/github/sherkhalil/COVID19/blob/master/GRU_classifier_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#path = '/content/drive/'
#p = path + 'My Drive/Thesis New Data/Data from Dr. Anwar Ali Yahya'
dataset = '/content/drive/My Drive/Jan9-2012-tweets-clean (version 1).csv'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pandas as pd 
data = pd.read_csv(dataset) 
data.head()

,Id,Tweet,Label
0,145353048817012000,Thinks that @melbahughes had a great 50th birt...,surprise
1,144279638024257000,"Como una expresiÃ³n tan simple, una sola oraci...",sadness
2,140499585285111000,the moment when you get another follower and y...,joy
3,145207578270507000,Be the greatest dancer of your life! practice ...,joy
4,139502146390470000,eww.. my moms starting to make her annual rum ...,disgust


In [5]:
data.Label.value_counts()

joy         8240
surprise    3849
sadness     3830
fear        2816
anger       1555
disgust      761
Name: Label, dtype: int64

In [6]:
data.drop(data.loc[data.Label == 'joy'].index, inplace=True)
data.drop(data.loc[data.Label == 'surprise'].index, inplace=True)

#data.drop(data.loc[data.Label == 'sadness'].index, inplace=True)
#data.drop(data.loc[data.Label == 'fear'].index, inplace=True)
#data.drop(data.loc[data.Label == 'anger'].index, inplace=True)
data.drop(data.loc[data.Label == 'disgust'].index, inplace=True)
data.Label.value_counts()

sadness    3830
fear       2816
anger      1555
Name: Label, dtype: int64

In [7]:
data['Label_Numeric'] = data['Label'].map({'sadness':0, 'fear':1,'anger':2})

In [8]:
data.Label_Numeric.value_counts()



0    3830
1    2816
2    1555
Name: Label_Numeric, dtype: int64

In [ ]:
# Import stopwords with nltk.
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
stop = stopwords.words('english')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:

data['AfterStopwords'] = data['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
import string
data['AfterPunctuation'] = data['AfterStopwords'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))

data['cleaned'] = data['AfterPunctuation'].str.lower()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

data['lemmatized'] = data['cleaned'].apply(lemmatize_text)


data["final1"]= data["lemmatized"].str.join(" ") 

data['final1'] = data['final1'].str.replace('\d+', '')

In [ ]:
data['final1'][100]

'gotta get  hour ride plane'

In [9]:
%tensorflow_version 1.x



from keras import backend as K
K.tensorflow_backend._get_available_gpus()


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen = 280
max_words = 40000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['Tweet'])
sequences = tokenizer.texts_to_sequences(data['Tweet'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data1 = pad_sequences(sequences, maxlen=maxlen)
print(data1)
def to_one_hot(labels, dimension=3):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
one_hot_labels = to_one_hot(data['Label_Numeric'])
labels = np.asarray(one_hot_labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data1.shape[0])
np.random.shuffle(indices)
data1 = data1[indices]
labels = labels[indices]
"""x_train = data1[:training_samples]
y_train = labels[:training_samples]
x_val = data1[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]
x_test = data1[validation_samples: validation_samples + test_samples]
y_test = labels[validation_samples: validation_samples + test_samples]"""

#from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data1, labels, test_size=0.1, random_state=0)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.03, random_state=0)

TensorFlow 1.x selected.


Using TensorFlow backend.


Found 17160 unique tokens.
[[    0     0     0 ...  6085  6086  1342]
 [    0     0     0 ...    31   133  2084]
 [    0     0     0 ...   281  6088  6089]
 ...
 [    0     0     0 ...    85    26  3403]
 [    0     0     0 ...    64     3 17158]
 [    0     0     0 ...  3514   986 17160]]
Shape of data tensor: (8201, 4)
Shape of label tensor: (8201, 3)


In [10]:
print(labels.shape)
print(y_test.shape)

(8201, 3)
(821, 3)


In [11]:
y_train.shape

(7380, 3)

In [14]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import Embedding, LSTM
from keras import regularizers
from keras import layers
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
embedding_dim = 300

model3 = Sequential()
model3.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model3.add(layers.GRU(32,return_sequences = True, dropout=0.2,recurrent_dropout=0.2))
model3.add(layers.GRU(32, return_sequences= True))
model3.add(layers.GlobalMaxPooling1D())
model3.add(layers.Dense(10, activation='relu'))
model3.add(layers.Dense(3, activation='softmax'))
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model3.summary()



history = model3.fit(X_train, y_train, epochs=10, batch_size=2048, validation_split = 0.1, shuffle=True)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 280, 300)          12000000  
_________________________________________________________________
gru_5 (GRU)                  (None, 280, 32)           31968     
_________________________________________________________________
gru_6 (GRU)                  (None, 280, 32)           6240      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 33        
Total params: 12,038,571
Trainable params: 12,038,571
Non-trainable params: 0
__________________________________________

In [15]:
from sklearn.metrics import precision_score , recall_score
y_val_pred=model3.predict_classes(X_test)

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


rounded_labels=np.argmax(y_test, axis=1)


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(rounded_labels, y_val_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
#precision = precision_score(rounded_labels, y_val_pred)
#print('Precision: %f' %precision)
# recall: tp / (tp + fn)
#recall = recall_score(rounded_labels, y_val_pred, average = 'weighted')
#print('Recall: %f' %recall)
# f1: 2 tp / (2 tp + fp + fn)
#f1 = f1_score(rounded_labels, y_val_pred)
#print('F1: %f' %f1)
# accuracy: (tp + tn) / (p + n)

Accuracy: 0.609013
